In [1]:
import pandas as pd
import mariadb
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from tqdm import tqdm
import bisect
import numpy as np


conn  = mariadb.connect(
    user='team3', 
    password='dbdb', 
    database='team3db', 
    host='localhost',
    port=3306
)

# 위 커넥션 정보와 동일하게 입력
engine = create_engine("mysql://{user}:{pw}@{host}/{db}".format(user='team3', pw='dbdb',host='localhost', db='team3db'))

In [2]:
select_sql = "SELECT * FROM train2"
train = pd.read_sql_query(select_sql, engine,index_col='index')
train.reset_index(inplace=True,drop=True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391934,TRAIN_391934,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,...,110.0,KEJZ24,"China, People's Republic Of",NaN,NaN,NaN,NaN,14,0.000552,0.000000
391935,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,80.0,JLTM64,Vietnam,-0.66,0.97,27.3,1.253491,8,0.002615,144.061389
391936,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,200.0,YERJ68,Singapore,-3.44,7.99,21.1,4.766257,18,0.000155,41.482222
391937,TRAIN_391937,TW,JWI3,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,...,170.0,HCZK58,Comoros,NaN,NaN,NaN,NaN,15,0.000990,7.485278


In [3]:
train = train.loc[train['ARI_PO']!='RGT8']

In [4]:
train.loc[train['ARI_PO']=='RGT8']

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR


In [5]:
train.drop(columns=['SAMPLE_ID','ARI_PO','V_WIND'],inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21012\421720492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
train

,ARI_CO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,SG,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,10.0,10.0,16700,180.0,CQSB78,Panama,NaN,NaN,NaN,5,0.002615,3.450000
1,IN,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,10.0,10.0,23500,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,12,0.000217,0.000000
2,CN,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,30.0,20.0,140000,370.0,FNPK22,Malta,NaN,NaN,NaN,6,0.001614,0.000000
3,JP,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,10.0,10.0,5400,120.0,PBZV77,Bahamas,-3.18,6.7,2.629350,13,0.000356,0.000000
4,SG,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,20.0,10.0,96600,300.0,GUCE76,Liberia,-0.33,25.6,2.495953,15,0.002615,253.554444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391934,JP,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,10.0,10.0,4820,110.0,KEJZ24,"China, People's Republic Of",NaN,NaN,NaN,14,0.000552,0.000000
391935,SG,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,10.0,10.0,1600,80.0,JLTM64,Vietnam,-0.66,27.3,1.253491,8,0.002615,144.061389
391936,US,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,20.0,10.0,34800,200.0,YERJ68,Singapore,-3.44,21.1,4.766257,18,0.000155,41.482222
391937,TW,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,10.0,10.0,16900,170.0,HCZK58,Comoros,NaN,NaN,NaN,15,0.000990,7.485278


In [7]:
train['ATA'] = pd.to_datetime(train['ATA'])

train['year'] = train['ATA'].dt.year
train['weekday'] = train['ATA'].dt.weekday

train.drop(columns='ATA', inplace=True)
train.shape

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21012\251422913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21012\251422913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21012\251422913.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

(391933, 21)

In [8]:
train = train.loc[train['year']!=2014]
train.shape

(387162, 21)

In [9]:
train = train.loc[train['year']!=2023]
train.shape

(376962, 21)

In [10]:
train.dropna(inplace=True)
train.shape

(217108, 21)

In [11]:
train.drop(columns=['ID', 'SHIPMANAGER', 'BN', 'ATA_LT'], inplace=True)
train.shape

(217108, 17)

In [12]:
train.reset_index(drop=True)

,ARI_CO,SHIP_TYPE_CATEGORY,DIST,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,FLAG,U_WIND,AIR_TEMPERATURE,PORT_SIZE,CI_HOUR,year,weekday
0,JP,Container,0.000000,20.0,18,6910,10.0,10.0,5400,120.0,Bahamas,-3.18,6.7,0.000356,0.000000,2020,4
1,SG,Container,27.037650,50.0,10,116000,20.0,10.0,96600,300.0,Liberia,-0.33,25.6,0.002615,253.554444,2020,6
2,AU,Bulk,49.953585,40.0,7,183000,20.0,20.0,94100,290.0,Japan,6.10,28.1,0.000103,68.391389,2021,4
3,TW,Cargo,0.000000,30.0,18,46600,20.0,10.0,29800,200.0,"Hong Kong, China",2.82,28.6,0.000990,0.000000,2022,4
4,CN,Bulk,18.022495,30.0,11,35000,20.0,10.0,22500,180.0,Montenegro,-2.16,25.6,0.000520,2.721944,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217103,AU,Bulk,55.601755,50.0,8,210000,20.0,20.0,107000,300.0,Liberia,6.97,24.9,0.000103,55.701667,2021,1
217104,SG,Container,7.304235,40.0,9,51700,20.0,10.0,41700,230.0,Singapore,-0.98,26.1,0.002615,23.914444,2019,1
217105,CN,Bulk,9.647703,30.0,26,69000,20.0,10.0,35900,220.0,Panama,3.64,21.4,0.000939,1095.597222,2020,0
217106,SG,Bulk,5.884603,10.0,12,3160,10.0,10.0,1600,80.0,Vietnam,-0.66,27.3,0.002615,144.061389,2019,2


In [13]:
q1, q3 = np.percentile(train['BUILT'],[25,75])
q1, q3

(9.0, 18.0)

In [14]:
upper_bound = q3 + ((q3-q1) * 1.5)
upper_bound

31.5

In [15]:
train.loc[train['BUILT']>30, 'BUILT'] = upper_bound

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21012\258716021.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '31.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


In [16]:
train.to_sql('train3',engine,if_exists='replace',chunksize=1000)

217108